### Introduction

This notebook is designed to help you determine if your map can be taken offline. It performs comprehensive checks on your layers, basemaps, and map properties to ensure that offline capabilities can be enabled. By using this notebook, you can identify specific issues and understand the logic behind each check. 

Enterprise users will find this notebook helpful since we introduced new offline checks for ArcGIS Online users in July 2024. Since Enterprise environments don't automatically receive AGOL updates, this notebook enables Enterprise users to implement equivalent validation flags, ensuring consistency with AGOL standards.

This process assumes that you have publishing and editing privileges for the map and that you are using either Enterprise 10.7+ or ArcGIS Online. This notebook uses python version 2.3.0.3

For further information, please refer to the official documentation: https://doc.arcgis.com/en/field-maps/latest/prepare-maps/configure-the-map.htm#ESRI_SECTION2_9DB2938BE8A749E393BBE43A3219E369


#### Import arcgis libraries

In [14]:
from arcgis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
from arcgis.mapping import MapServiceLayer
from arcgis.mapping import VectorTileLayer

#### Access your GIS Account -ArcGIS Online
After you run this cell, you will be prompted for your user name and password. 

In [ ]:
# gis = GIS("home")
print("Enter your ArcGIS account user name: ")
username = input()
gis = GIS("agol_url", username)
print("Connected to {}".format(gis.properties.portalHostname))

#### Access your GIS Account -ArcGIS Enterprise
If you are using an enterprise account use the cell below to initialize this notebook. Just add in your portal url.

In [16]:
# enterprise_url = ""
# print('Enter your ArcGIS account user name: ')
# username = input()
# gis = GIS(enterprise_url, username)
# print('Connected to {}'.format(gis.properties.portalHostname))

#### Add in your Map
In the `map_item_id_to_check` variable below there is an example map id. Please delete this and add in the id of your map you would like to check for offline compatibility

In [17]:
MAP_ITEM_ID_TO_CHECK = "YOUR MAP ID"
offline_map = gis.content.get(MAP_ITEM_ID_TO_CHECK)

#### Checking Basemaps for Offline Use
In this step, we will perform essential checks to ensure that basemaps are suitable for offline use. We will evaluate four key aspects:

1. Exportable Map Service Layer: Verify if the map service layer is exportable.
2. Pre-approved Tile Layers: Check if the tile layers are from a pre-approved list that can be exported.
3. Export Tiles Enabled: Ensure that vector tile or tile map service layers have export tiles enabled, as this is required for offline use.
4. Multi Sourced Basemaps: Ensure that your basemap does not have more than one source
5. Depcrecated Tile Base Maps URLs: Ensure you are not using basemaps with base url: `https://tiledbasemaps.arcgis.com/arcgis/rest/services`

By performing these checks, we can ensure that the basemaps are properly configured and optimized for offline use.

In [18]:
def is_export_enabled_map_service_layer(basemaplayer) -> bool:
    try:
        msl = MapServiceLayer(basemaplayer["url"])
        return msl.properties.exportTilesAllowed
    except Exception as e:
        if "Token Required (Error Code: 499)" in str(e):
            print("You must Proxy your basemap with AGOL")
            return False


def is_exportable_agol_tile_layer(basemaplayer) -> bool:
    lowercase_layer_url = basemaplayer["url"].lower()
    host_list = ["server.arcgisonline.com", "services.arcgisonline.com"]
    has_allowed_host_name = any(
        host_str in lowercase_layer_url for host_str in host_list
    )
    service_list = [
        "natgeo_world_map",
        "ocean_basemap",
        "usa_topo_maps",
        "world_imagery",
        "world_street_map",
        "world_terrain_base",
        "world_topo_map",
        "world_hillshade",
        "canvas/world_light_gray_base",
        "canvas/world_light_gray_reference",
        "canvas/world_dark_gray_base",
        "canvas/world_dark_gray_reference",
        "ocean/world_ocean_base",
        "ocean/world_ocean_reference",
        "reference/world_boundaries_and_places",
        "reference/world_reference_overlay",
        "reference/world_transportation",
    ]

    has_exportable_agol_servicename = any(
        service_name in lowercase_layer_url for service_name in service_list
    )

    return has_allowed_host_name and has_exportable_agol_servicename


def is_export_enabled_vector_tile_layer(basemaplayer) -> bool:
    try:
        if "itemId" in basemaplayer:
            vl_item = basemaplayer["itemId"]
            vtl = VectorTileLayer.fromitem(vl_item)
            return vtl.properties.exportTilesAllowed
        elif "styleUrl" in basemaplayer:
            vtl = VectorTileLayer(basemaplayer["styleUrl"])
            vtl_source = VectorTileLayer(vtl.properties.sources.esri.url)
            return vtl_source.properties.exportTilesAllowed
    except Exception as e:
        if "Token Required (Error Code: 499)" in str(e):
            print("You must Proxy your basemap with AGOL")
            return False


def is_export_enabled(layer) -> bool:
    layer_type = layer["layerType"]
    if layer_type == "ArcGISTiledMapServiceLayer":
        if is_export_enabled_map_service_layer(layer):
            return True
        if not is_exportable_agol_tile_layer(layer):
            return False
        return True

    elif layer_type == "VectorTileLayer":
        if not is_export_enabled_vector_tile_layer(layer):
            return False
        return True

In [19]:
def is_map_multisource(basemaplayer) -> bool:
    if "styleUrl" in basemaplayer:
        vtl = VectorTileLayer(basemaplayer["styleUrl"])
        return len(vtl.properties.sources) > 1
    return False

In [20]:
deprecated_urls = [
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/NatGeo_World_Map/MapServer",
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/USA_Topo_Maps/MapServer",
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/Reference/World_Boundaries_and_Places/MapServer",
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/Canvas/World_Dark_Gray_Base/MapServer",
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/Canvas/World_Dark_Gray_Reference/MapServer",
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/Canvas/World_Light_Gray_Base/MapServer",
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/Canvas/World_Light_Gray_Reference/MapServer",
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/Ocean/World_Ocean_Reference/MapServer",
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/Reference/World_Reference_Overlay/MapServer",
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/World_Street_Map/MapServer",
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/World_Topo_Map/MapServer",
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/Reference/World_Transportation/MapServer",
    "https://tiledbasemaps.arcgis.com/arcgis/rest/services/World_Terrain_Base/MapServer",
]


def is_deprecated_basemap_urls(basemap) -> bool:
    if basemap["layerType"] == "ArcGISTiledMapServiceLayer":
        return basemap.get("url") in deprecated_urls
    return False

In [21]:
def check_basemap(basemaps) -> str:
    basemap_errors = []
    for basemap in basemaps["baseMapLayers"]:
        basemap_title = basemap["title"]
        if not is_export_enabled(basemap):
            basemap_errors.append(f"{basemap_title} export tiles not enabled")
        else:
            if is_map_multisource(basemap):
                basemap_errors.append(f"{basemap_title} is multisource")
            if is_deprecated_basemap_urls(basemap):
                basemap_errors.append(f"{basemap_title} is depcrecated")
    return basemap_errors

#### Layer Checks for Offline Use
##### In this section, we perform 8 essential checks to ensure that layers are suitable for offline use:

1. Layer Type Support: Verify if the layer type is supported for offline use.
2. Sync Enabled: Ensure that sync is enabled, as it is required for offline use.
3. Supported Indices: Check if the layer's indices are supported, ensuring they do not contain + or -.
4. Join View Check: Determine if the layer is a join view, as join view layers are not suitable for offline use.
5. Global ID Presence: Verify if the layer has a global ID, as its absence can cause issues.
6. Relationship Keywords: Check if relationship keywords are missing, which are necessary for certain functionalities.
7. Subtype Fields: Ensure that subtype fields are present, as their absence can affect layer behavior.
8. True Curve Updates: Confirm that True Curve Updates are set to true, as required.

By performing these checks, we can ensure that the layers are properly configured and optimized for offline use.

In [22]:
def is_supported_layer_type(layer) -> bool:
    if "featureCollectionType" in layer and (
        layer["featureCollectionType"] == "route"
        or layer["featureCollectionType"] == "notes"
    ):
        return False

    layer_type = layer["layerType"] if "layerType" in layer else layer.layerType
    return layer_type in [
        "ArcGISFeatureLayer",
        "GroupLayer",
        "ArcGISImageServiceLayer",
        "ArcGISTiledImageServiceLayer",
        "ArcGISTiledMapServiceLayer",
        "SubtypeGroupLayer",
        "VectorTileLayer",
    ]

In [23]:
def is_sync_enabled(layer) -> bool:
    capabilities = layer.properties.capabilities.split(",")
    if "Sync" in capabilities:
        return True
    else:
        return False

In [24]:
def is_supported_feature_layer_indices(layer) -> bool:
    for index in layer.properties.indexes:
        if "+" in index.name or "-" in index.name:
            return False
        else:
            return True

In [25]:
def is_join_view(layer) -> bool:
    layer_properties = layer.properties
    if "isView" in layer_properties and layer_properties["isView"]:
        if (
            "isMultiServicesView" in layer_properties
            and layer_properties["isMultiServicesView"]
        ):
            return True
    return False

In [26]:
def is_global_id_missing(layer) -> bool:
    layer_properties = layer.properties
    if "globalIdField" in layer_properties:
        return layer_properties["globalIdField"] == ""
    return False

In [27]:
def is_relationship_missing(layer) -> bool:
    layer_properties = layer.properties
    layer_fields = layer.properties.fields
    if (
        "relationships" in layer_properties
        and len(layer_properties["relationships"]) > 0
    ):
        relationship_key = layer_properties["relationships"][0]["keyField"].lower()
        filtered_relationship_fields = [
            field for field in layer_fields if relationship_key in field.name.lower()
        ]
        return len(filtered_relationship_fields) == 0
    return False

In [28]:
def is_subtype_missing(layer) -> bool:
    layer_properties = layer.properties
    layer_fields = layer.properties.fields
    if "subtypes" in layer_properties and layer_properties["subtypes"] is not None:
        if (
            len(layer_properties["subtypes"]) > 0
            or len(layer_properties["subtypeField"]) > 0
        ):
            subtype_field = layer_properties["subtypeField"].lower()
            filtered_subtype_fields = [
                field for field in layer_fields if subtype_field in field.name.lower()
            ]
            return len(filtered_subtype_fields) == 0
    return False

In [29]:
def is_true_curve_ready(layer) -> bool:
    layer_properties = layer.properties
    if (
        "allowTrueCurvesUpdates" in layer_properties
        and layer_properties["allowTrueCurvesUpdates"] == True
    ):
        if (
            "onlyAllowTrueCurveUpdatesByTrueCurveClients" in layer_properties
            and layer_properties["onlyAllowTrueCurveUpdatesByTrueCurveClients"] == True
        ):
            return False
    if (
        "allowTrueCurvesUpdates" in layer_properties
        and layer_properties["allowTrueCurvesUpdates"] == False
    ):
        if (
            "onlyAllowTrueCurveUpdatesByTrueCurveClients" in layer_properties
            and layer_properties["onlyAllowTrueCurveUpdatesByTrueCurveClients"] == False
        ):
            return False
    else:
        return True

In [30]:
def check_tile_vector_operational_layers(layer) -> bool:
    if not is_export_enabled(layer):
        return False
    else:
        if is_map_multisource(layer):
            return False
        if is_deprecated_basemap_urls(layer):
            return False
    return True

#### Field Checks for Offline Use
In this section, we perform essential checks to ensure that fields within layers are suitable for offline use:

1. Field Type Support: Verify if the field type is supported. The following field types are not supported:

        `esriFieldTypeBigInteger`

        `esriFieldTypeDateOnly`

        `esriFieldTypeTimeOnly`

        `esriFieldTypeTimestampOffset` 

If any field has one of these types, it is not supported for offline use.

2. Field Name Length: Ensure that the field name does not exceed 31 characters. Field names longer than 31 characters are not supported.
3. Forbidden SQL Keywords: Check if the field name is a forbidden SQL keyword. Field names that are SQL keywords are not supported.

By performing these checks, we can ensure that the fields within layers are properly configured and optimized for offline use.

In [31]:
forbidden_sql_keywords = [
    "add",
    "all",
    "alter",
    "and",
    "as",
    "autoincrement",
    "between",
    "case",
    "cast",
    "check",
    "collate",
    "commit",
    "constraint",
    "create",
    "default",
    "deferrable",
    "delete",
    "distinct",
    "drop",
    "else",
    "escape",
    "except",
    "exists",
    "foreign",
    "from",
    "group",
    "having",
    "in",
    "index",
    "insert",
    "intersect",
    "into",
    "is",
    "isnull",
    "join",
    "limit",
    "not",
    "nothing",
    "notnull",
    "null",
    "on",
    "or",
    "order",
    "primary",
    "raise",
    "references",
    "returning",
    "select",
    "set",
    "table",
    "then",
    "to",
    "transaction",
    "union",
    "unique",
    "update",
    "using",
    "values",
    "when",
    "where",
]

In [32]:
def is_supported_feature_layer_field(fields) -> bool:
    for field in fields:
        if field["type"] in [
            "esriFieldTypeBigInteger",
            "esriFieldTypeDateOnly",
            "esriFieldTypeTimeOnly",
            "esriFieldTypeTimestampOffset",
        ]:
            return False

        if len(field["name"]) > 31:
            return False

        if field["name"] in forbidden_sql_keywords:
            return False
        else:
            return True

#### Checking All Layers and Tables in Your Map

##### In this step, we will call each of the layer and field check methods defined earlier to evaluate the suitability of the layers for offline use. 

The results of these checks will be recorded, and a list will be created to identify any errors.

1. Invoke Layer and Field Checks: We systematically call each of the predefined layer and field check methods for every layer in the map.

2. Record Results: The results of these checks are recorded, capturing both the boolean outcome and any associated messages.


By following this process, we can efficiently identify and document any issues that need to be addressed to ensure the layers are suitable for offline use

In [33]:
def check_layers(layers) -> str:
    layer_errors = []
    for op_layer in layers:
        if not is_supported_layer_type(op_layer):
            layer_errors.append(f"Layer type not supported: {op_layer.title}")
            continue

        elif op_layer["layerType"] == "ArcGISFeatureLayer":
            layer_url = op_layer["url"] if "url" in op_layer else op_layer.url
            layer_title = op_layer["title"] if "title" in op_layer else op_layer.title
            feature_layer = FeatureLayer(layer_url)
            if not is_sync_enabled(feature_layer):
                layer_errors.append(f"{layer_title} is sync not enabled")
            if not is_supported_feature_layer_field(feature_layer.properties.fields):
                layer_errors.append(f"{layer_title} has field name errors")
            if not is_supported_feature_layer_indices(feature_layer):
                layer_errors.append(f"{layer_title} has unsupported layer indicies")
            if is_join_view(feature_layer):
                layer_errors.append(f"{layer_title} is a join view")
            if is_global_id_missing(feature_layer):
                layer_errors.append(f"{layer_title} has globalid missing")
            if is_relationship_missing(feature_layer):
                layer_errors.append(
                    f"{layer_title} has missing relationship field names"
                )
            if is_subtype_missing(feature_layer):
                layer_errors.append(f"{layer_title} has missing subtype field")
            if not is_true_curve_ready(feature_layer):
                layer_errors.append(f"{layer_title} is not true curve enabled")
            # add more tests here
        elif op_layer["layerType"] == "GroupLayer":
            layer_errors.extend(check_layers(op_layer["layers"]))
        elif op_layer["layerType"] in [
            "ArcGISImageServiceLayer",
            "ArcGISTiledImageServiceLayer",
            "ArcGISTiledMapServiceLayer",
            "VectorTileLayer",
        ]:
            if not check_tile_vector_operational_layers(op_layer):
                layer_errors.append("Your operational tile layer has errors")

    layer_errors = list(dict.fromkeys(layer_errors))

    return layer_errors

In [34]:
def check_tables(tables, table_errors=[], table_urls=[]) -> str:
    for op_table in tables:
        if not is_supported_layer_type(op_table):
            table_errors.append(f"Layer type not supported: {op_table.title}")
            continue

        elif op_table["layerType"] == "ArcGISFeatureLayer":
            table_url = op_table["url"] if "url" in op_table else op_table.url
            table_title = op_table["title"] if "title" in op_table else op_table.title
            table_urls.append(table_url)
            feature_layer = FeatureLayer(table_url)
            if not is_sync_enabled(feature_layer):
                table_errors.append(f"{table_title} is sync not enabled")
            if not is_supported_feature_layer_indices(feature_layer):
                table_errors.append(f"{table_title} has unsupported layer indicies")
            if not is_supported_feature_layer_field(feature_layer.properties.fields):
                table_errors.append(f"{table_title} has field name errors")
            if is_join_view(feature_layer):
                table_errors.append(f"{table_title} is a join view")
            if is_global_id_missing(feature_layer):
                table_errors.append(f"{table_title} has globalid missing")
            if is_relationship_missing(feature_layer):
                table_errors.append(
                    f"{table_title} has missing relationship field names"
                )
            if not is_true_curve_ready(feature_layer):
                table_errors.append(f"{table_title} is not true curve enabled")
            # add more tests here

    table_errors = list(dict.fromkeys(table_errors))

    return table_errors

#### Map Checks

The Map checks section checks if generic map properties are capable of going offline.
These checks include if the map has the Offline Disabled type keyword in the map or if the map has any duplicate layers.

In [35]:
def check_offline_typekeyword(map) -> bool:
    if "OfflineDisabled" in map.typeKeywords:
        return False
    if "Offline" in map.typeKeywords:
        return True

In [36]:
def map_has_duplicates(map) -> bool:
    webmap_object = WebMap(map)
    item_urls = set()
    for layer in webmap_object.layers + webmap_object.tables:
        if is_supported_layer_type(layer):
            if layer["layerType"] in ["ArcGISFeatureLayer", "featureCollectionType"]:
                item_urls.add(layer["url"] if "url" in layer else layer.url)
            elif layer["layerType"] == "GroupLayer":
                for sub_layer in layer["layers"]:
                    item_urls.add(
                        sub_layer["url"] if "url" in sub_layer else sub_layer.url
                    )
    return len(item_urls) != len(webmap_object.layers + webmap_object.tables)

In [37]:
def check_map(webmap) -> str:
    map_errors = []
    webmap_title = webmap.title
    if not check_offline_typekeyword(webmap):
        map_errors.append(f"{webmap_title} is offline disabled")
    if map_has_duplicates(webmap):
        map_errors.append(f"{webmap_title} has duplicate layers")
    return map_errors

#### Check WebMap Offline Compatability general method
This method is designed to return to you what parts of your map has any errors in it -- layers, tables, basemap or the map itself

In [38]:
def check_webmap_offline_compatability(webmap):
    webmap_object = WebMap(webmap)
    offline_compatibility = {
        "map": [],
        "layers": [],
        "tables": [],
        "basemap": [],
        "has_errors": False,
    }

    offline_compatibility["map"] = check_map(webmap)
    offline_compatibility["layers"] = check_layers(webmap_object.layers)
    offline_compatibility["tables"] = check_tables(webmap_object.tables)
    offline_compatibility["basemap"] = check_basemap(webmap_object.basemap)

    if (
        offline_compatibility["map"]
        or offline_compatibility["layers"]
        or offline_compatibility["tables"]
        or offline_compatibility["basemap"]
    ):
        offline_compatibility["has_errors"] = True

    return offline_compatibility

#### Lets see the results
In section we call and use everything we have defined above. The layer or basemap that is preventing you from going offline will be printed below along with the appropriate error.

In [39]:
offline_compatability = check_webmap_offline_compatability(offline_map)
if offline_compatability["has_errors"]:
    for key, values in offline_compatability.items():
        if key != "has_errors":
            print(f"{key}:")
            for value in values:
                print(f"  - {value}")
else:
    "Your map is ready for offline!"

map:
  - offlinetest is offline disabled
layers:
  - Hydrants Just Subtype is sync not enabled
  - Join Features to All_Layer_Errors_Offline_R2 view is sync not enabled
  - Join Features to All_Layer_Errors_Offline_R2 view is a join view
  - Join Features to All_Layer_Errors_Offline_R2 view has globalid missing
  - Join Features to All_Layer_Errors_Offline_R2 view has missing subtype field
tables:
basemap:
  - USA Topo Maps (for Export) export tiles not enabled
